In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import CosineRecommender
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix

In [2]:
data_dir = "/Users/ur001/Documents/Datasets/lastfm-dataset-360K"
dest_data_dir = "/Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest2"

In [37]:
def load_data(path): 
    return pd.read_table(
        os.path.join(path, "usersha1-artmbid-artname-plays.tsv"),
        usecols=[0, 1, 2, 3],
        names=['user', 'artist', 'artist_name', 'plays'],
        dtype={'user': str, 'artist': str, 'artist_name': str, 'plays': np.float32}
    )

def prepare_data(data):
    """Возвращает подготовленные данные и словарь артистов"""
    # Заменяем отсутствующие значения идентификаторов исполнителя на его имя
    print("Filling na artists...")
    empty_artist = data.artist.isnull()
    data.loc[empty_artist, 'artist'] = data.loc[empty_artist, 'artist_name']
    data.dropna(inplace=True)
    
    # Преобразуем пользователей и исполнителей в числа
    print("Encoding artits and users...")
    data['artist_id'] = data.artist.astype("category").cat.codes.copy() + 1
    data['user_id'] = data.user.astype("category").cat.codes.copy() + 1
    
    # Составляем словарь исполнителей
    print("Saving atists dict...")
    artists = dict(data.groupby('artist_id').artist_name.first())
    return data[['user_id', 'artist_id', 'plays']], artists

In [31]:
from sklearn.model_selection import train_test_split

def split_data(data, size=0.2):
    data_train, data_test = train_test_split(data, test_size=size)
    test_user_set = set(data_test.user_id.unique())
    train_user_set = set(data_train.user_id.unique())
    # Оставляем только пользователей которые есть одновременно в тестовой и обучающей выборке
    user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
    return (
        data_train[~data_train.user_id.isin(user_ids_to_exclude)], 
        data_test[~data_test.user_id.isin(user_ids_to_exclude)]
    )

In [32]:
def save_data(path, name, data, columns=["user_id", "artist_id", "plays"]):
    file_name = os.path.join(path, name + ".tsv")
    data[columns].to_csv(file_name, sep="\t", header=False, index=False)

In [33]:
def make_coo_matrix(data):
    """Создаёт разреженную матрицу item*user"""
    return coo_matrix((
        data.plays.astype(np.double),
        (data.artist_id, data.user_id)
    ))

def sparse_info(sparse_matrix):
    """функция, которая красиво печатает информацию о разреженных матрицах"""
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}".format(
        sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1]
    ))
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [34]:
def make_user_recommendations(model, data_train_coo, data_test, path, model_name):
    """Генерирует рекомендации для каждого пользователя и сохраняет их в файл для проверки mrec-ом"""
    user_plays = data_train_coo.T.tocsr()
    file_name = "test.{}.tsv.recs.tsv".format(model_name)
    # Получаем столько рекомендаций, сколько записей у пользователя в тестовой выборке, но не меньше 3
    user_rec_counts = dict(data_test.user_id.value_counts())
    user_rec_counts = {user_id: max(10, int(N)) for user_id, N in user_rec_counts.items()}
    with open(os.path.join(path, file_name), "w") as output_file:
        for user_id in tqdm_notebook(data_test.user_id.unique()):
            for artist_id, score in model.recommend(user_id, user_plays, N=user_rec_counts[user_id]):
                output_file.write("{}\t{}\t{}\n".format(user_id, artist_id, score))    

In [35]:
def search_artist(artists, name_prefix):
    """Поиск по базе исполнителей: выводит список всех начинающихся с переданного префикса"""
    for artist_id, artist_name in artists.items():
        if artist_name.lower().startswith(name_prefix):
            print("[{}]\t{}".format(artist_id, artist_name))
            
def find_similar_artists(model, artist_id, artists, count=10):
    """Выводит список похожих исполнителей"""
    print("Similar to {}:".format(artists[artist_id]))
    print("-------------------------------")
    for similar_artist_id, score in model.similar_items(artist_id, count):
        print("{:.3f}\t{}".format(score, artists[similar_artist_id]))

In [57]:
data = load_data(data_dir)

In [58]:
data, artists = prepare_data(data)

Filling na artists...
Encoding artits and users...
Saving atists dict...


In [59]:
data.head()

,user_id,artist_id,plays
0,1,39710,2137.0
1,1,211106,1099.0
2,1,135575,897.0
3,1,40727,717.0
4,1,143199,706.0


In [60]:
data_train, data_test = split_data(data, 0.2)
data_train.shape, data_test.shape

((14024955, 3), (3507120, 3))

In [61]:
data_train.head()

,user_id,artist_id,plays
4225467,86456,45124,16.0
11025302,225591,111125,96.0
10901689,223064,106864,20.0
12543339,256682,88187,31.0
535852,10975,31767,40.0


In [62]:
save_data(dest_data_dir, 'train', data_train)

In [63]:
data_train_coo = make_coo_matrix(data_train)

In [64]:
sparse_info(data_train_coo)

Размерности матрицы: (268591, 358869)
Ненулевых элементов в матрице: 14024955
Доля ненулевых элементов: 0.00014550371296845563
Среднее значение ненулевых элементов: 215.2549501941361
Максимальное значение ненулевых элементов: 419157.0
Минимальное значение ненулевых элементов: 0.0


In [65]:
# строим матрицу схожести по косинусной мере
models = {}
models['knn_cosine'] = CosineRecommender()
models['knn_cosine'].fit(data_train_coo)

In [66]:
save_data(dest_data_dir, 'test.knn_cosine', data_test)

In [67]:
# Попробуем найти похожих исполнителей на Aphex Twin, чтобы убедиться что рекомендации вообще работают
search_artist(artists, 'aphex twin')

[128603]	aphex twin & céline eia
[128604]	aphex twin & m-ziq (aka rich & mike)
[128605]	aphex twin & squarepusher
[128606]	aphex twin & µ-ziq (mike & rich)
[128607]	aphex twin & µ-ziq - mike and rich
[128608]	aphex twin (aka afx)
[128609]	aphex twin (aka caustic window)
[128610]	aphex twin (aka polygon window)
[128611]	aphex twin (aka universal indicator)
[128612]	aphex twin - power-pill
[128613]	aphex twin ft. photek & autechre
[128614]	aphex twin vs luke vibert
[128615]	aphex twin vs run jeremy
[128616]	aphex twin/afx
[128617]	aphex twin/luke vibert
[128618]	aphex twin/richard james
[210593]	aphex twin


In [68]:
find_similar_artists(models['knn_cosine'], 210593, artists)

Similar to aphex twin:
-------------------------------
1.000	aphex twin
0.442	afx
0.358	magnat
0.353	gak
0.348	flare
0.302	polygon window
0.291	the tuss
0.235	autechre
0.230	squarepusher
0.227	boards of canada


In [69]:
# Создаём рекомендации для всех тестовых пользователей
make_user_recommendations(models['knn_cosine'], data_train_coo, data_test, dest_data_dir, 'knn_cosine')

HBox(children=(IntProgress(value=0, max=358509), HTML(value='')))

In [70]:
!/Users/ur001/.pyenv/versions/netology1/bin/mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest2/test.knn_cosine.tsv \
    --recsdir /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest2
    
# mrr            0.1170 +/- 0.0000
# prec@5         0.0427 +/- 0.0000
# prec@10        0.0345 +/- 0.0000
# prec@15        0.0252 +/- 0.0000
# prec@20        0.0190 +/- 0.0000 

[2017-12-04 20:52:48,514] INFO: processing /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest2/test.knn_cosine.tsv...
None
mrr            0.1162 +/- 0.0000
prec@5         0.0421 +/- 0.0000
prec@10        0.0342 +/- 0.0000
prec@15        0.0250 +/- 0.0000
prec@20        0.0189 +/- 0.0000


In [72]:
# Тренируем ALS-модель
models['als'] = AlternatingLeastSquares(factors=64)
models['als'].fit(data_train_coo)

In [73]:
find_similar_artists(models['als'], 210593, artists)

Similar to aphex twin:
-------------------------------
1.000	aphex twin
0.923	boards of canada
0.875	squarepusher
0.873	autechre
0.834	air
0.819	massive attack
0.807	portishead
0.794	dj shadow
0.778	amon tobin
0.778	björk


In [78]:
make_user_recommendations(models['als'], data_train_coo, data_test, dest_data_dir, 'als')

In [76]:
save_data(dest_data_dir, 'test.als', data_test)

In [77]:
!/Users/ur001/.pyenv/versions/netology1/bin/mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest2/test.als.tsv \
    --recsdir /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest2
    
# mrr            0.3151 +/- 0.0000
# prec@5         0.1410 +/- 0.0000
# prec@10        0.1176 +/- 0.0000
# prec@15        0.0860 +/- 0.0000
# prec@20        0.0651 +/- 0.0000    

[2017-12-04 21:42:51,704] INFO: processing /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest2/test.als.tsv...
None
mrr            0.3151 +/- 0.0000
prec@5         0.1410 +/- 0.0000
prec@10        0.1176 +/- 0.0000
prec@15        0.0860 +/- 0.0000
prec@20        0.0651 +/- 0.0000
